## Problem Summary

To replicate the Deep Structural Causal Models for Tractable Counterfactual Inference[1]paper , and apply it to google cartoon faces dataset[3] and answer counterfactual queries on the same. 

We aim to explicitly model causal relationships with a fully specified causal models with no unobserved confounding and inferring exogenous noise via  normalising flows.

Our goal is to validate our causal assumptions; if our causal assumptions are valid, these simulations should align with our imagination. 



In [2]:
!pip install -q torch pyro-ppl 

     |████████████████████████████████| 678 kB 7.1 MB/s 


In [17]:
from pyro.distributions.conditional import ConditionalTransformModule
from pyro.distributions.torch_transform import TransformModule
from pyro.distributions import transforms as pyro_transforms
from torch.distributions import transforms 
import torch
from torch.distributions.utils import lazy_property
from torch.distributions import constraints
from torch.distributions.transforms import Transform
import numpy as np

## Standard libraries
import os
import math
import time
import numpy as np 
import pandas as pd 

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import set_matplotlib_formats 
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()

## Progress bar
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install pytorch-lightning==1.3.4
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint 

# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

# Fetching the device that will be used throughout this notebook
device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda:0")
print("Using device", device)

from torch.multiprocessing import Pool, Process, set_start_method
try:
     set_start_method('spawn')
except RuntimeError:
    pass

import torch 
import pyro.distributions as dist 
from torch.distributions.normal import Normal 
from torch.distributions.uniform import Uniform 
from torch.distributions.beta import Beta 
from scipy import stats 

import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 

from pyro.nn import PyroModule, pyro_method 
import torch
import pyro
from pyro.nn import PyroModule, pyro_method
from pyro.distributions import Normal, TransformedDistribution
from pyro.distributions.torch_transform import ComposeTransformModule
from pyro.distributions.conditional import ConditionalTransformedDistribution
from pyro.distributions.transforms import (
    Spline, ExpTransform, ComposeTransform, ConditionalAffineCoupling,
    GeneralizedChannelPermute, SigmoidTransform
    )
from pyro.distributions.transforms import ComposeTransform, SigmoidTransform, AffineTransform 
from pyro.nn import DenseNN 
import PIL 

Global seed set to 42


Using device cpu


In [4]:
from google.colab import drive 
drive.mount("/content/drive", force_remount=False)
! cp -r /content/drive/My\ Drive/data/cartoon-faces/filtered_data_500.npy ./ 
# ! cp -r /content/drive/My\ Drive/data/cartoon-faces/cartoon_resized.npy ./ 
! cp -r /content/drive/My\ Drive/data/cartoon-faces/filtered_data_500.csv ./ 

Mounted at /content/drive


## Data Generation

We use the google cartoon dataset to train our model. The original 4D (10 k) dataset was transformed into lower dimension grayscale dataset due to computation resource limitation. 

Also, for simplicity we convert the categorical feature into binary (glasses/no glasses)

![Dataset Snapshot](./images/cartoon_snapshot.png)

In [5]:
def load_data(path_1,path_2):
    """Open the folder for cartoon dataset and combine them into one dataset with added column flenames which stores
    the corresponding png filename
    :param path_1: File path to the cartoon dataset directory
    :param path_2: File path to save the cobined csv file
    :returns: saves the combined file to the given path_2"""

    import os
    import glob
    import pandas as pd

    os.chdir(path_1)
    #os.chdir("Documents/GitHub/causalfairness/cartoonset10k/")
    extension = 'csv'
    all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

    # combine all files in the list
    combined_csv = pd.DataFrame()
    for f in all_filenames:
        df = pd.read_csv(f)
        df = df.T
        df = df.reset_index()
        df = df.drop([0,2])
        df['filename'] = f
        df = df.reset_index()
        combined_csv = combined_csv.append(df,ignore_index=True)

    # export to csv
    combined_csv = combined_csv.drop(columns=['level_0'])
    combined_csv.columns = ['eye_angle','eye_lashes','eye_lid','chin_length','eyebrow_weight','eyebrow_shape','eyebrow_thickness','face_shape','facial_hair','hair','eye_color','face_color','hair_color','glasses','glasses_color','eye_slant','eyebrow_width','eye_eyebrow_distance','filename']
    combined_csv.to_csv(path_2+"combined_csv.csv", index=False, encoding='utf-8-sig')
    return

def columntobinary(path_1,path_2):
    """Open the combined data file and make a filtered binary datafile
    :param path_1: File path to the combined file
    :param path_2: File path to save binary file
    :returns: saves the binary file to path_2"""
    import pandas as pd
    df = pd.read_csv(path_1)
    df["facial_hair"].replace({14: 0}, inplace=True)
    df["glasses"].replace({11: 0}, inplace=True)
    df.to_csv(path_2+"filtered_data_binary_new.csv", index=False, encoding='utf-8-sig')

    print(df)
    return

In [6]:
def grayscale(arr): 
    arr = arr.astype(np.uint8) 
    arr = PIL.Image.fromarray(arr).resize(size=(64,64)).convert("L")
    arr = np.expand_dims(np.array(arr), axis=0) 
    return torch.Tensor(arr) 

In [8]:
cartoon_features = pd.read_csv("filtered_data_500.csv") 
cartoon_features = cartoon_features.drop(["filename"], axis=1) 
cartoon_features = cartoon_features.values 

In [9]:
cartoon = np.load("filtered_data_500.npy") 
cartoon_resized = list(map(lambda x: grayscale(x), cartoon)) 
cartoon_resized = np.concatenate(cartoon_resized, axis=0) 

cartoon_data = list(map(lambda x, y: (torch.Tensor(np.expand_dims(x, axis=0)).to(device), torch.Tensor(y).to(device)), cartoon_resized, cartoon_features))  

pl.seed_everything(42) 
train_loader = data.DataLoader(cartoon_data, batch_size=256, shuffle=False, drop_last=False) 

Global seed set to 42


## Model Pieces Explained

#### Normalizing Flows


Normalizing flows are a family of methods which allows for constructing more flexible probability distributions, commonly learned using neural networks. 

The path traversed by the random variables is the flow and the full chain formed by the successive distributions   is called a normalizing flow. 

Required by the computation in the equation, a transformation function  should satisfy two properties:

    1. It is easily invertible.
    2. Its Jacobian determinant is easy to compute.


#### Normalizing Flows on Images

Build a normalizing flow that maps an input image  to an equally sized latent space. 

Training and Validation: Perform density estimation in the forward direction by applying  a series of flow transformations on the input 𝑥 and estimate the probability of the input by determining the probability of the transformed point  𝑧  given a prior, and the change of volume caused by the transformations. 

Inference:  density estimation and sample new points by inverting the flow transformations


![Normalizing flows on Images](./images/nf_images.png)

#### Coupling Layers

A popular flow layer which lends itself to the architecture of neural networks is the coupling layer

A given input z is split into two parts, the first part is passed through unchanged while the second part has a function dependent on both parts applied to it

A standard version is the affine transformation, implemented as: $z_{1:j}'' = z_{1:j}, z_{j+1,d}' = \mu(z_{1:j}) + \sigma(z_{1:j})*z_{j+1:d}$ 

![Coupling Layers](./images/coupling_layers.png)

### Custom transforms (not available in torch or pyro) 

### Squeeze Transform

In [18]:
class SqueezeTransform(Transform):
    """A transformation defined for image data that trades spatial dimensions for channel
    dimensions, i.e. "squeezes" the inputs along the channel dimensions.
    Implementation adapted from https://github.com/pclucas14/pytorch-glow and
    https://github.com/chaiyujin/glow-pytorch.
    Reference:
    > L. Dinh et al., Density estimation using Real NVP, ICLR 2017.
    """

    domain = constraints.real
    codomain = constraints.real
    bijective = True
    event_dim = 3
    volume_preserving = True

    def __init__(self, factor=2):
        super().__init__(cache_size=1)

        self.factor = factor

    def _call(self, inputs):
        """
        :param x: the input into the bijection
        :type x: torch.Tensor
        Invokes the bijection x=>y; in the prototypical context of a
        :class:`~pyro.distributions.TransformedDistribution` `x` is a sample from
        the base distribution (or the output of a previous transform)
        """
        if inputs.dim() < 3:
            raise ValueError(f'Expecting inputs with at least 3 dimensions, got {inputs.shape} - {inputs.dim()}')

        *batch_dims, c, h, w = inputs.size()
        num_batch = len(batch_dims)

        if h % self.factor != 0 or w % self.factor != 0:
            raise ValueError('Input image size not compatible with the factor.')

        inputs = inputs.view(*batch_dims, c, h // self.factor, self.factor, w // self.factor,
                             self.factor)
        permute = np.array((0, 2, 4, 1, 3)) + num_batch
        inputs = inputs.permute(*np.arange(num_batch), *permute).contiguous()
        inputs = inputs.view(*batch_dims, c * self.factor * self.factor, h // self.factor,
                             w // self.factor)

        return inputs

    def _inverse(self, inputs):
        """
        :param y: the output of the bijection
        :type y: torch.Tensor
        Inverts y => x.
        """
        if inputs.dim() < 3:
            raise ValueError(f'Expecting inputs with at least 3 dimensions, got {inputs.shape}')

        *batch_dims, c, h, w = inputs.size()
        num_batch = len(batch_dims)

        if c < 4 or c % 4 != 0:
            raise ValueError('Invalid number of channel dimensions.')

        inputs = inputs.view(*batch_dims, c // self.factor ** 2, self.factor, self.factor, h, w)
        permute = np.array((0, 3, 1, 4, 2)) + num_batch
        inputs = inputs.permute(*np.arange(num_batch), *permute).contiguous()
        inputs = inputs.view(*batch_dims, c // self.factor ** 2, h * self.factor, w * self.factor)

        return inputs

    def log_abs_det_jacobian(self, x, y):
        """
        Calculates the elementwise determinant of the log Jacobian, i.e.
        log(abs([dy_0/dx_0, ..., dy_{N-1}/dx_{N-1}])). Note that this type of
        transform is not autoregressive, so the log Jacobian is not the sum of the
        previous expression. However, it turns out it's always 0 (since the
        determinant is -1 or +1), and so returning a vector of zeros works.
        """

        log_abs_det_jacobian = torch.zeros(x.size()[:-3], dtype=x.dtype, layout=x.layout, device=x.device)
        return log_abs_det_jacobian

    def get_output_shape(self, c, h, w):
        return (c * self.factor * self.factor,
                h // self.factor,
                w // self.factor)

#### Reshape Transform

In [ ]:
class ReshapeTransform(Transform):
    domain = constraints.real
    codomain = constraints.real
    bijective = True
    volume_preserving = True

    def __init__(self, input_shape, output_shape):
        super().__init__()
        # self.event_dim = len(input_shape)
        self.inv_event_dim = len(output_shape)
        self.input_shape = input_shape
        self.output_shape = output_shape

    def _call(self, inputs):
        """
        :param x: the input into the bijection
        :type x: torch.Tensor
        Invokes the bijection x=>y; in the prototypical context of a
        :class:`~pyro.distributions.TransformedDistribution` `x` is a sample from
        the base distribution (or the output of a previous transform)
        """
        batch_dims = inputs.shape[:-self.event_dim]
        inp_shape = inputs.shape[-self.event_dim:]
        if inp_shape != self.input_shape:
            raise RuntimeError('Unexpected inputs shape ({}, but expecting {})'
                               .format(inp_shape, self.input_shape))
        return inputs.reshape(*batch_dims, *self.output_shape)

    def _inverse(self, inputs):
        """
        :param y: the output of the bijection
        :type y: torch.Tensor
        Inverts y => x.
        """
        batch_dims = inputs.shape[:-self.inv_event_dim]
        inp_shape = inputs.shape[-self.inv_event_dim:]
        if inp_shape != self.output_shape:
            raise RuntimeError('Unexpected inputs shape ({}, but expecting {})'
                               .format(inp_shape, self.output_shape))
        return inputs.reshape(*batch_dims, *self.input_shape)

    def log_abs_det_jacobian(self, x, y):
        """
        Calculates the elementwise determinant of the log Jacobian, i.e.
        log(abs([dy_0/dx_0, ..., dy_{N-1}/dx_{N-1}])). Note that this type of
        transform is not autoregressive, so the log Jacobian is not the sum of the
        previous expression. However, it turns out it's always 0 (since the
        determinant is -1 or +1), and so returning a vector of zeros works.
        """

        log_abs_det_jacobian = torch.zeros(x.size()[:-3], dtype=x.dtype, layout=x.layout, device=x.device)
        return log_abs_det_jacobian 

#### Transpose Transform

In [ ]:
class TransposeTransform(Transform):
    """
    A bijection that reorders the input dimensions, that is, multiplies the input by
    a permutation matrix. This is useful in between
    :class:`~pyro.distributions.transforms.AffineAutoregressive` transforms to
    increase the flexibility of the resulting distribution and stabilize learning.
    Whilst not being an autoregressive transform, the log absolute determinate of
    the Jacobian is easily calculable as 0. Note that reordering the input dimension
    between two layers of
    :class:`~pyro.distributions.transforms.AffineAutoregressive` is not equivalent
    to reordering the dimension inside the MADE networks that those IAFs use; using
    a :class:`~pyro.distributions.transforms.Permute` transform results in a
    distribution with more flexibility.
    Example usage:
    >>> from pyro.nn import AutoRegressiveNN
    >>> from pyro.distributions.transforms import AffineAutoregressive, Permute
    >>> base_dist = dist.Normal(torch.zeros(10), torch.ones(10))
    >>> iaf1 = AffineAutoregressive(AutoRegressiveNN(10, [40]))
    >>> ff = Permute(torch.randperm(10, dtype=torch.long))
    >>> iaf2 = AffineAutoregressive(AutoRegressiveNN(10, [40]))
    >>> flow_dist = dist.TransformedDistribution(base_dist, [iaf1, ff, iaf2])
    >>> flow_dist.sample()  # doctest: +SKIP
    :param permutation: a permutation ordering that is applied to the inputs.
    :type permutation: torch.LongTensor
    """

    domain = constraints.real
    codomain = constraints.real
    bijective = True
    volume_preserving = True

    def __init__(self, permutation):
        # self.event_dim = len(permutation) 
        self.permutation = permutation

        super().__init__(cache_size=1) 

    @lazy_property
    def inv_permutation(self):
        result = torch.empty_like(self.permutation, dtype=torch.long)
        result[self.permutation] = torch.arange(self.permutation.size(0),
                                                dtype=torch.long,
                                                device=self.permutation.device)
        return result

    def _call(self, x):
        """
        :param x: the input into the bijection
        :type x: torch.Tensor
        Invokes the bijection x=>y; in the prototypical context of a
        :class:`~pyro.distributions.TransformedDistribution` `x` is a sample from
        the base distribution (or the output of a previous transform)
        """

        *batch_dims, c, h, w = x.size()
        num_batch = len(batch_dims)

        return x.permute(*np.arange(num_batch), *(self.permutation + num_batch)).contiguous()

    def _inverse(self, y):
        """
        :param y: the output of the bijection
        :type y: torch.Tensor
        Inverts y => x.
        """

        *batch_dims, c, h, w = y.size()
        num_batch = len(batch_dims)

        return y.permute(*np.arange(num_batch), *(self.inv_permutation + num_batch)).contiguous()

    def log_abs_det_jacobian(self, x, y):
        """
        Calculates the elementwise determinant of the log Jacobian, i.e.
        log(abs([dy_0/dx_0, ..., dy_{N-1}/dx_{N-1}])). Note that this type of
        transform is not autoregressive, so the log Jacobian is not the sum of the
        previous expression. However, it turns out it's always 0 (since the
        determinant is -1 or +1), and so returning a vector of zeros works.
        """

        log_abs_det_jacobian = torch.zeros(x.size()[:-3], dtype=x.dtype, layout=x.layout, device=x.device)
        return log_abs_det_jacobian


class LearnedAffineTransform(TransformModule, transforms.AffineTransform):
    def __init__(self, loc=None, scale=None, **kwargs):

        super().__init__(loc=loc, scale=scale, **kwargs)

        if loc is None:
            self.loc = torch.nn.Parameter(torch.zeros([1, ]))
        if scale is None:
            self.scale = torch.nn.Parameter(torch.ones([1, ]))

    def _broadcast(self, val):
        dim_extension = tuple(1 for _ in range(val.dim() - 1))
        loc = self.loc.view(-1, *dim_extension)
        scale = self.scale.view(-1, *dim_extension)

        return loc, scale

    def _call(self, x):
        loc, scale = self._broadcast(x)

        return loc + scale * x

    def _inverse(self, y):
        loc, scale = self._broadcast(y)
        return (y - loc) / scale

#### Conditional Affine Transform

In [ ]:
class ConditionalAffineTransform(ConditionalTransformModule):
    def __init__(self, context_nn, event_dim=0, **kwargs):
        super().__init__(**kwargs)

        self.event_dim = event_dim
        self.context_nn = context_nn

    def condition(self, context):
        loc, log_scale = self.context_nn(context)
        scale = torch.exp(log_scale)

        ac = transforms.AffineTransform(loc, scale, event_dim=self.event_dim)
        return ac


class ActNorm(TransformModule):
    codomain = constraints.real
    bijective = True
    event_dim = 3

    def __init__(self, features):
        """
        Transform that performs activation normalization. Works for 2D and 4D inputs. For 4D
        inputs (images) normalization is performed per-channel, assuming BxCxHxW input shape.
        Reference:
        > D. Kingma et. al., Glow: Generative flow with invertible 1x1 convolutions, NeurIPS 2018.
        """
        super().__init__()

        self.initialized = False
        self.log_scale = nn.Parameter(torch.zeros(features))
        self.shift = nn.Parameter(torch.zeros(features))

    @property
    def scale(self):
        return torch.exp(self.log_scale)

    def _broadcastable_scale_shift(self, inputs):
        if inputs.dim() == 4:
            return self.scale.view(1, -1, 1, 1), self.shift.view(1, -1, 1, 1)
        else:
            return self.scale.view(1, -1), self.shift.view(1, -1)

    def _call(self, x):
        if x.dim() not in [2, 4]:
            raise ValueError("Expecting inputs to be a 2D or a 4D tensor.")

        if self.training and not self.initialized:
            self._initialize(x)

        scale, shift = self._broadcastable_scale_shift(x)
        outputs = scale * x + shift

        return outputs

    def _inverse(self, y):
        if y.dim() not in [2, 4]:
            raise ValueError("Expecting inputs to be a 2D or a 4D tensor.")

        scale, shift = self._broadcastable_scale_shift(y)
        outputs = (y - shift) / scale

        return outputs

    def log_abs_det_jacobian(self, x, y):
        """
        Calculates the elementwise determinant of the log Jacobian, i.e.
        log(abs([dy_0/dx_0, ..., dy_{N-1}/dx_{N-1}])). Note that this type of
        transform is not autoregressive, so the log Jacobian is not the sum of the
        previous expression. However, it turns out it's always 0 (since the
        determinant is -1 or +1), and so returning a vector of zeros works.
        """

        ones = torch.ones(x.shape[0], device=x.device)
        if x.dim() == 4:
            _, _, h, w = x.shape
            log_abs_det_jacobian = h * w * torch.sum(self.log_scale) * ones
        else:
            log_abs_det_jacobian = torch.sum(self.log_scale) * ones

        return log_abs_det_jacobian

    def _initialize(self, inputs):
        """Data-dependent initialization, s.t. post-actnorm activations have zero mean and unit
        variance. """
        if inputs.dim() == 4:
            num_channels = inputs.shape[1]
            inputs = inputs.permute(0, 2, 3, 1).reshape(-1, num_channels)

        with torch.no_grad():
            std = inputs.std(dim=0)
            mu = (inputs / std).mean(dim=0)
            self.log_scale.data = -torch.log(std)
            self.shift.data = -mu

        self.initialized = True

#### Image transforms architecture 

In [19]:
import torch
from torch import nn
import numpy as np
from collections.abc import Iterable


class BasicFlowConvNet(nn.Module):
    def __init__(self, in_channels: int, hidden_channels: int, param_dims, context_dims: int = None, param_nonlinearities=None):
        super().__init__()
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels

        self.param_dims = param_dims
        self.count_params = len(param_dims)
        self.output_dims = sum(param_dims)

        self.context_dims = context_dims
        self.param_nonlinearities = param_nonlinearities

        self.seq1 = nn.Sequential(
            nn.Conv2d(in_channels + context_dims if context_dims is not None else in_channels, hidden_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, hidden_channels, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, self.output_dims, kernel_size=3, padding=1)
        )

        ends = torch.cumsum(torch.tensor(param_dims), dim=0)
        starts = torch.cat((torch.zeros(1).type_as(ends), ends[:-1]))
        self.param_slices = [slice(s.item(), e.item()) for s, e in zip(starts, ends)]

        def weights_init(m):
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight.data, 0., 1e-4)
                if m.bias is not None:
                    nn.init.constant_(m.bias.data, 0.)

        self.apply(weights_init)

    def forward(self, inputs, context=None):
        # pyro affine coupling splits on the last dimenion and not the channel dimension
        # -> we want to permute the dimensions to move the last dimension into the channel dimension
        # and then transpose back

        if not ((self.context_dims is None) == (context is None)):
            raise ValueError('Given context does not match context dims: context: {} and context_dims:{}'.format(context, self.context_dims))

        *batch_dims, h, w, c = inputs.size() 
        num_batch = len(batch_dims)

        permutation = np.array((2, 0, 1)) + num_batch
        outputs = inputs.permute(*np.arange(num_batch), *permutation).contiguous() 

        if context is not None:
            # assuming scalar inputs [B, C] 
            """
            torch.Size([16, 16, 8])
            torch.Size([1, 1, 16])
            Ouptus torch.Size([1, 8, 16, 16])
            """
            context = context.view(*context.shape, 1, 1).expand(-1, *outputs.shape[1:]) 
            outputs = torch.cat([outputs, context], 0)
            if num_batch == 0: 
                outputs = outputs.view(-1, *outputs.shape) 

        outputs = self.seq1(outputs) 
        if num_batch == 0: 
            outputs = outputs.view(*outputs.shape[1:]) 

        permutation = np.array((1, 2, 0)) + num_batch
        outputs = outputs.permute(*np.arange(num_batch), *permutation).contiguous()

        if self.count_params > 1:
            outputs = tuple(outputs[..., s] for s in self.param_slices)

        if self.param_nonlinearities is not None:
            if isinstance(self.param_nonlinearities, Iterable):
                outputs = tuple(n(o) for o, n in zip(outputs, self.param_nonlinearities))
            else:
                outputs = tuple(self.param_nonlinearities(o) for o in outputs)

        return outputs

## Full Model

**Structural Causal Model**

($e_g$, $e_x$) ~ N(0, 1)

g ~ Bern($e_g$)

X ~ $F(e_x, g)$

![SCM](./images/scm.png)

![Full Model](./images/x_transforms.png)

**Multi-Scale Architecture**

Disadvantage of normalizing flows is that they operate on the exact same dimensions as the input.
    
    If the input is high-dimensional, so is the latent space,  will requires larger computational cost to learn suitable transformations. 
    
    However, in image domain, many pixels contain less information. 

Multi-scale architecture : After the first 𝑁  flow transformations, we split off half of the latent dimensions and directly evaluate them on the prior. The other half is run through  𝑁  more flow transformations, and depending on the size of the input, we split it again in half or stop overall at this position. 

    Squeeze and split:

![Squeeze and Split](./images/squeeze_split.png)

In [20]:
class FlowSCM(PyroModule):
    def __init__(self, use_affine_ex=True):
        super().__init__() 
        self.num_scales = 2
        self.flows_per_scale = 1
        self.use_actnorm = False 
        self.use_affine_ex = use_affine_ex 

        self.register_buffer("glasses_base_loc", torch.zeros([1, ], requires_grad=False))
        self.register_buffer("glasses_base_scale", torch.ones([1, ], requires_grad=False))

        self.register_buffer("glasses_flow_lognorm_loc", torch.zeros([], requires_grad=False))
        self.register_buffer("glasses_flow_lognorm_scale", torch.ones([], requires_grad=False))

        self.glasses_flow_lognorm = AffineTransform(loc=self.glasses_flow_lognorm_loc.item(), scale=self.glasses_flow_lognorm_scale.item())

        self.glasses_flow_components = ComposeTransformModule([Spline(1)])
        self.glasses_flow_constraint_transforms = ComposeTransform([self.glasses_flow_lognorm,
            SigmoidTransform()])
        self.glasses_flow_transforms = ComposeTransform([self.glasses_flow_components,
            self.glasses_flow_constraint_transforms])

        glasses_base_dist = Normal(self.glasses_base_loc, self.glasses_base_scale).to_event(1)
        self.glasses_dist = TransformedDistribution(glasses_base_dist, self.glasses_flow_transforms)
        
        self._build_image_flow()
        self.register_buffer("x_base_loc", torch.zeros([1, 64, 64], requires_grad=False))
        self.register_buffer("x_base_scale", torch.ones([1, 64, 64], requires_grad=False))
        self.x_base_dist = Normal(self.x_base_loc, self.x_base_scale).to_event(3)

    def model(self): 
        glasses_ = pyro.sample("glasses_", self.glasses_dist)
        glasses = pyro.sample("glasses", dist.Bernoulli(glasses_))
        glasses_context = self.glasses_flow_constraint_transforms.inv(glasses_)

        cond_x_transforms = ComposeTransform(
            ConditionalTransformedDistribution(self.x_base_dist, self.x_transforms)
            .condition(glasses_context).transforms
            ).inv
        cond_x_dist = TransformedDistribution(self.x_base_dist, cond_x_transforms)
        x = pyro.sample("x", cond_x_dist)

        return x, glasses 


    def sample(self, n_samples=1):
        with pyro.plate('observations', n_samples):
            samples = self.model()

        return (*samples,)


    def _build_image_flow(self):
        self.trans_modules = ComposeTransformModule([])
        self.x_transforms = [] 
        self.hidden_channels = 3 

        c = 1
        for _ in range(self.num_scales):
            self.x_transforms.append(SqueezeTransform())
            c *= 4

            for _ in range(self.flows_per_scale):
                if self.use_actnorm:
                    actnorm = ActNorm(c)
                    self.trans_modules.append(actnorm)
                    self.x_transforms.append(actnorm)

                gcp = GeneralizedChannelPermute(channels=c)
                self.trans_modules.append(gcp)
                self.x_transforms.append(gcp)

                self.x_transforms.append(TransposeTransform(permutation=torch.tensor((1, 2, 0))))

                ac = ConditionalAffineCoupling(c // 2, BasicFlowConvNet(c // 2, self.hidden_channels, (c // 2, c // 2), 1))
                self.trans_modules.append(ac)
                self.x_transforms.append(ac)

                self.x_transforms.append(TransposeTransform(torch.tensor((2, 0, 1))))

            gcp = GeneralizedChannelPermute(channels=c)
            self.trans_modules.append(gcp)
            self.x_transforms.append(gcp)

        self.x_transforms += [
            ReshapeTransform((4**self.num_scales, 64 // 2**self.num_scales, 64 // 2**self.num_scales), (1, 64, 64))
        ]

        if self.use_affine_ex:
            affine_net = DenseNN(1, [16, 16], param_dims=[1, 1])
            affine_trans = ConditionalAffineTransform(context_nn=affine_net, event_dim=3)

            self.trans_modules.append(affine_trans)
            self.x_transforms.append(affine_trans) 

**Model Training**

**Bits Per Dimension (BPD)**

As a final piece for calculating our loss function we use a concept called Bits Per Dimension (BPD)

This loss calculates the number of bits needed to represent some sample x’ in our distribution P(X), with less bits corresponding to a larger likelihood

We change the base of the log likelihood to base 2 and then divide by the product over the dimensions of our image (which is the width and height)

Essentially we normalize over the dimensions we have over our images to allow for comparison between images of varying resolutions 

    This is important as we change the image resolutions to help speed up training


In [21]:
class Experiment(pl.LightningModule): 
    def __init__(self, model): 
        super().__init__() 
        self.flow_model = model 
        self.model = flow_model.model 

    def configure_optimizers(self):
        glasses_params = self.flow_model.glasses_flow_components.parameters()
        x_params = self.flow_model.trans_modules.parameters() 

        optimizer =  torch.optim.Adam([
            {'params': x_params, 'lr': 1e-3},
            {'params': glasses_params, 'lr': 1e-3},
        ], lr=1e-3, eps=1e-5) 
        scheduler = optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.99)
        return [optimizer], [scheduler]
    
    def _loss(self, **batch): 
        cond_model = pyro.condition(self.flow_model.sample, data=batch) 
        model_trace = pyro.poutine.trace(cond_model).get_trace(batch["x"].shape[0]) 
        model_trace.compute_log_prob() 

        log_probs = {}
        nats_per_dim = {}
        for name, site in model_trace.nodes.items():
            if site["type"] == "sample" and site["is_observed"]:
                log_probs[name] = site["log_prob"].mean()
                log_prob_shape = site["log_prob"].shape
                value_shape = site["value"].shape
                if len(log_prob_shape) < len(value_shape):
                    dims = np.prod(value_shape[len(log_prob_shape):])
                else:
                    dims = 1.
                nats_per_dim[name] = -site["log_prob"].mean() / dims
                if self.hparams.validate:
                    print(f'at site {name} with dim {dims} and nats: {nats_per_dim[name]} and logprob: {log_probs[name]}')
                    if torch.any(torch.isnan(nats_per_dim[name])):
                        raise ValueError('got nan')

        return log_probs, nats_per_dim 


    def prep_batch(self, batch):
        x = batch[0].float() 
        context = batch[1][:,2] 
        return {"x": x, "glasses": context} 


    def training_step(self, batch, *args): 
        batch = self.prep_batch(batch) 
        log_probs, nats_per_dim = self._loss(**batch) 
        loss = torch.stack(tuple(nats_per_dim.values())).sum() 

        return loss 

In [ ]:
trainer = pl.Trainer(default_root_dir="./", 
                         gpus=0 if torch.cuda.is_available() else 0, 
                         max_epochs=100, 
                         gradient_clip_val=1.0,
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode="min", monitor="val_bpd"),
                                    LearningRateMonitor("epoch")])
trainer.logger._log_graph = True
trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

train_data_loader = data.DataLoader(cartoon_data, batch_size=64, shuffle=True, drop_last=True, pin_memory=True, num_workers=8)
result = None 

flow_model = FlowSCM() 
exp = Experiment(flow_model)
trainer.fit(exp, train_loader) 

In [23]:
flow_model.x_transforms

[SqueezeTransform(),
 GeneralizedChannelPermute(),
 TransposeTransform(),
 ConditionalAffineCoupling(
   (nn): BasicFlowConvNet(
     (seq1): Sequential(
       (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       (1): ReLU()
       (2): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
       (3): ReLU()
       (4): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     )
   )
 ),
 TransposeTransform(),
 GeneralizedChannelPermute(),
 SqueezeTransform(),
 GeneralizedChannelPermute(),
 TransposeTransform(),
 ConditionalAffineCoupling(
   (nn): BasicFlowConvNet(
     (seq1): Sequential(
       (0): Conv2d(9, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       (1): ReLU()
       (2): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
       (3): ReLU()
       (4): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     )
   )
 ),
 TransposeTransform(),
 GeneralizedChannelPermute(),
 ReshapeTransform(),
 ConditionalAffineTransform(
   

**Results**

**References**

1. Pawlowski, N., Castro, D. C., & Glocker, B. (2020). Deep structural causal models for tractable counterfactual inference. arXiv preprint arXiv:2006.06485.
2. Normalizing Flows - Introduction (Part 1) — Pyro Tutorials 1.7.0 documentation
3. Cartoon Dataset 
4. Normalizing Flows for image modeling
5. Dinh, L., Sohl-Dickstein, J., and Bengio, S. (2017). “Density estimation using Real NVP,” In: 5th International Conference on Learning Representations, ICLR 2017.
6. Ho, J., Chen, X., Srinivas, A., Duan, Y., and Abbeel, P. (2019). “Flow++: Improving Flow-Based Generative Models with Variational Dequantization and Architecture Design,” in Proceedings of the 36th International Conference on Machine Learning, vol. 97, pp. 2722–2730
7. Flow-based Deep Generative Models
